# Sinvular Value Decomposition (SVD)-1

In this tutorial, we show how to perform
* SVD for a rank-n tensor.
* SVD + truncation for a rank-2 tensor.

## API change
* `N_rowrank-->rowrank`.

In [1]:
import Tor10
import copy

def Tprint(T):
    print(T.Print_diagram())
    print(T)

## SVD for a rank-4 tensor.

In [2]:
bd = Tor10.Bond(2)
T = Tor10.UniTensor([bd, bd, bd, bd], rowrank=2, name='T', labels=[10,11,12,13])
T.Rand()
Tprint(T)

-----------------------
tensor Name : T
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    10 ____| 2         2 |____ 12 
           |             |     
    11 ____| 2         2 |____ 13 
           \             /     
            -------------      
None
Tensor name: T
is_diag    : False
tensor([[[[0.4315, 0.4814],
          [0.9250, 0.5966]],

         [[0.9679, 0.5767],
          [0.4183, 0.3819]]],


        [[[0.0466, 0.0426],
          [0.5829, 0.4267]],

         [[0.2143, 0.2102],
          [0.5423, 0.3379]]]], dtype=torch.float64)



### Step-1: Convert $T$ into a rank-2 tensor $M$ via `GetBlock`

In [3]:
M = T.GetBlock()
M.SetName('M')
Tprint(M)

-----------------------
tensor Name : M
tensor Rank : 2
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 4         4 |____ 1  
           \             /     
            -------------      
None
Tensor name: M
is_diag    : False
tensor([[0.4315, 0.4814, 0.9250, 0.5966],
        [0.9679, 0.5767, 0.4183, 0.3819],
        [0.0466, 0.0426, 0.5829, 0.4267],
        [0.2143, 0.2102, 0.5423, 0.3379]], dtype=torch.float64)



### Step-2: Perform SVD on $M$

* The results are rank-2 tensors.

In [4]:
# three identical ways to perform SVD
UM, S, VtM = M.Svd()
UM, S, VtM = Tor10.Svd(M)
UM, S, VtM = Tor10.linalg.Svd(M)

UM.SetName('UM')
S.SetName('S')
VtM.SetName('VtM')
UM.Print_diagram()
S.Print_diagram()
VtM.Print_diagram()

-----------------------
tensor Name : UM
tensor Rank : 2
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 4         4 |____ -1 
           \             /     
            -------------      
-----------------------
tensor Name : S
tensor Rank : 2
has_symmetry: False
on device     : cpu
is_diag       : True
            -------------      
           /             \     
    -1 ____| 4         4 |____ -2 
           \             /     
            -------------      
-----------------------
tensor Name : VtM
tensor Rank : 2
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    -2 ____| 4         4 |____ 1  
           \             /     
            -------------      


### Step-3: Obtain the SVD of $T$ via `PutBlock`

* Create rank-k tensor $U$ and $Vt$ that are compatible with $UM$ and $VtM$.
* Use `PutBlock` to copy the values of $UM (VtM)$ tensor into $U (Vt)$ tensor.
* The **left** bonds of $U$ are the same as the left bonds of $T$.
* The **right** bonds of $U$ are the same as the right bonds of $UM$.
* The **left** bonds of $Vt$ are the same as the left bonds of $VtM$.
* The **right** bonds of $Vt$ are the same as the right bonds of $T$.
* It is a good idea to also copy the labels.



In [5]:
Tprint(UM)
U = Tor10.UniTensor([T.bonds[0], T.bonds[1], UM.bonds[1]], 
                    labels=[T.labels[0], T.labels[1], UM.labels[1]], rowrank=2, name='U', )
U.PutBlock(UM)
Tprint(U)

-----------------------
tensor Name : UM
tensor Rank : 2
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 4         4 |____ -1 
           \             /     
            -------------      
None
Tensor name: UM
is_diag    : False
tensor([[-0.6529,  0.2929,  0.5901, -0.3737],
        [-0.5906, -0.7529, -0.2900, -0.0160],
        [-0.3136,  0.5398, -0.7510, -0.2149],
        [-0.3557,  0.2365,  0.0604,  0.9022]], dtype=torch.float64)

-----------------------
tensor Name : U
tensor Rank : 3
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    10 ____| 2         4 |____ -1 
           |             |     
    11 ____| 2           |        
           \             /     
            -------------      
None
Tensor name: U
is_diag    : False
tensor([[[-0.6529,  0.2929,  0.5901, -0.3737],
         [-0.5906, -0.7529, -0.2900, 

In [6]:
Vt = Tor10.UniTensor([VtM.bonds[0], T.bonds[2], T.bonds[3]], 
                     labels=[VtM.labels[0], T.labels[2], T.labels[3]], rowrank=1, name='Vt')
Vt.PutBlock(VtM)
Tprint(Vt)

-----------------------
tensor Name : Vt
tensor Rank : 3
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    -2 ____| 4         2 |____ 12 
           |             |     
           |           2 |____ 13 
           \             /     
            -------------      
None
Tensor name: Vt
is_diag    : False
tensor([[[-0.4906, -0.3861],
         [-0.6374, -0.4516]],

        [[-0.7274, -0.3046],
         [ 0.5511,  0.2728]],

        [[-0.3843,  0.7797],
         [ 0.1559, -0.4692]],

        [[ 0.2872, -0.3877],
         [ 0.5154, -0.7082]]], dtype=torch.float64)



In [7]:
# Check T = U S Vt
Tprint(T - Tor10.Contract(Tor10.Contract(U, S), Vt))

-----------------------
tensor Name : 
tensor Rank : 4
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
    10 ____| 2         2 |____ 12 
           |             |     
    11 ____| 2         2 |____ 13 
           \             /     
            -------------      
None
Tensor name: 
is_diag    : False
tensor([[[[ 2.7756e-16,  5.5511e-16],
          [-1.1102e-16,  2.2204e-16]],

         [[ 3.3307e-16,  2.2204e-16],
          [ 3.3307e-16,  3.3307e-16]]],


        [[[ 1.1102e-16,  4.1633e-17],
          [-1.1102e-16,  0.0000e+00]],

         [[ 1.1102e-16,  8.3267e-17],
          [ 0.0000e+00,  0.0000e+00]]]], dtype=torch.float64)



### Get the same result via `Reshape`

In [8]:
UMp = UM.Reshape([2, 2, 4], rowrank=2)
Tprint(UMp - U)

VtMp = VtM.Reshape([4, 2, 2], rowrank=1)
Tprint(VtMp - Vt)

-----------------------
tensor Name : 
tensor Rank : 3
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 2         4 |____ 2  
           |             |     
     1 ____| 2           |        
           \             /     
            -------------      
None
Tensor name: 
is_diag    : False
tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.]]], dtype=torch.float64)

-----------------------
tensor Name : 
tensor Rank : 3
has_symmetry: False
on device     : cpu
is_diag       : False
            -------------      
           /             \     
     0 ____| 4         2 |____ 1  
           |             |     
           |           2 |____ 2  
           \             /     
            -------------      
None
Tensor name: 
is_diag    : False
tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.